In [1]:
import os
import pandas as pd
import torch
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pydicom
import numpy as np
from PIL import Image

In [ ]:
data_path = "data/"
train_root = os.path.join(data_path, "stage_2_train_images")

df = pd.read_csv(os.path.join(data_path, "stage_2_train_labels.csv"))

df_pneumonia = df[df["Target"] == 1]

# 바운딩 박스 좌표 리스트
df_grouped = df_pneumonia.groupby("patientId").agg({
    "x": list,
    "y": list,
    "width": list,
    "height": list
}).reset_index()

train_files, test_files = train_test_split(df_grouped["patientId"].tolist(), test_size=0.2, random_state=55) # tolist() 리스트 변환
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)

train_df = df_grouped[df_grouped["patientId"].isin(train_files)] # isin() 일치하는지 확인
val_df = df_grouped[df_grouped["patientId"].isin(val_files)]
test_df = df_grouped[df_grouped["patientId"].isin(test_files)]

print(f"폐렴 Train 데이터 개수: {len(train_df)}")
print(f"폐렴 Val 데이터 개수: {len(val_df)}")
print(f"폐렴 Test 데이터 개수: {len(test_df)}")

In [3]:
def load_dicom_image(dicom_path):
    dicom_data = pydicom.dcmread(dicom_path)
    image = dicom_data.pixel_array.astype("uint8") # 8비트로 변환

    image = Image.fromarray(image).convert("RGB")

    imagenet_transform = transforms.Compose([
        transforms.Resize((600, 600)), # 논문 권장 사이즈
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    return imagenet_transform(image)

In [4]:
def adjust_bbox(bbox, orig_size=(1024, 1024), new_size=(600, 600)):
    """
    Bounding Box를 원본 크기에서 리사이즈된 크기에 맞게 변환하며, 좌표를 x1, y1, x2, y2로 변환.
    """
    x_scale = new_size[0] / orig_size[0]
    y_scale = new_size[1] / orig_size[1]

    x1 = bbox[0] * x_scale
    y1 = bbox[1] * y_scale
    x2 = (bbox[0] + bbox[2]) * x_scale  # width → x2 변환
    y2 = (bbox[1] + bbox[3]) * y_scale  # height → y2 변환

    # 좌표값이 음수가 되지 않도록 보정
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = max(x1 + 1, x2)  # x2가 x1보다 작아지는 것 방지
    y2 = max(y1 + 1, y2)  # y2가 y1보다 작아지는 것 방지

    return [x1, y1, x2, y2]  # Faster R-CNN 형식으로 반환

In [5]:
class PneumoniaDetectionDataset(Dataset):
    def __init__(self, df, img_dir):
        self.df = df
        self.img_dir = img_dir

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        dicom_path = os.path.join(self.img_dir, f"{row['patientId']}.dcm")
        image = load_dicom_image(dicom_path)

        # Bounding Box 변환 및 필터링
        boxes = torch.tensor([
            adjust_bbox([row["x"][i], row["y"][i], row["width"][i], row["height"][i]]) 
            for i in range(len(row["x"]))
        ], dtype=torch.float32)

        # 너비(height)와 높이가 0이 아닌 경우만 유지
        valid_boxes = [box for box in boxes if (box[2] > box[0] and box[3] > box[1])]

        # 유효한 Bounding Box가 없는 경우 샘플 삭제
        if len(valid_boxes) == 0:
            return self.__getitem__((idx + 1) % len(self))  # 다음 인덱스 샘플을 반환

        valid_boxes = torch.stack(valid_boxes) if valid_boxes else torch.zeros((0, 4), dtype=torch.float32)
        labels = torch.ones((len(valid_boxes),), dtype=torch.int64)

        return image, {"boxes": valid_boxes, "labels": labels}


In [ ]:
dataset = PneumoniaDetectionDataset(train_df, "data/stage_2_train_images")

# 데이터셋 크기 확인
print(f"Dataset size: {len(dataset)}")

# 첫 번째 데이터 샘플 로드
image, target = dataset[0]

# 이미지 텐서의 크기 확인
print(f"Image Shape: {image.shape}")  # (C, H, W)

# Bounding Box 정보 확인
print(f"Bounding Boxes Shape: {target['boxes'].shape}")  # (num_boxes, 4)
print(f"Labels: {target['labels']}")  # Class labels

# Bounding Box 좌표 출력
print(f"Bounding Boxes:\n{target['boxes'].numpy()}")  # Tensor → NumPy 변환 후 출력


In [ ]:
train_dataset = PneumoniaDetectionDataset(train_df, train_root)
val_dataset = PneumoniaDetectionDataset(val_df, train_root)
test_dataset = PneumoniaDetectionDataset(test_df, train_root)

def collate_fn(batch):
    images, targets = zip(*batch)  # 이미지와 타겟을 분리
    return list(images), list(targets)  # 리스트 형태로 변환하여 반환

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

print(f"Train 데이터 개수: {len(train_dataset)}")
print(f"Validation 데이터 개수: {len(val_dataset)}")
print(f"Test  데이터 개수: {len(test_dataset)}")

In [ ]:
# dataset 체크

import pydicom
from PIL import Image

sample_image, sample_label = train_dataset[0]

print(f"Image Shape: {sample_image.shape}")
print(f"Label: {sample_label}")

In [ ]:
# dataloader 체크

batch = next(iter(train_loader))
batch_images, batch_labels = batch

print(f"Batch Image Shape: {batch_images[0].shape}")
print(f"Batch Labels: {batch_labels}")

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 2  # 배경과 폐렴 클래스를 구분
in_features = model.roi_heads.box_predictor.cls_score.in_features # cls_score.in_features는 FC Layer의 입력 뉴런 개수
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [11]:
# import torch.optim as optim
# from tqdm import tqdm

# optimizer = optim.AdamW(model.parameters(), lr=0.0001)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

# num_epochs = 100
# best_val_loss = float('inf')
# early_stopping_patience = 10
# early_stopping_counter = 0

# for epoch in range(num_epochs):
#     print(f"\nEpoch [{epoch+1}/{num_epochs}] 시작\n")

#     # Train Step
#     model.train()
#     total_train_loss = 0
#     total_train_correct = 0
#     total_train_samples = 0

#     train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)
#     for images, targets in train_loader_tqdm:
#         images = [img.to(device) for img in images]
#         targets = [{k: v.to(device) for k, v in target.items()} for target in targets] 

#         optimizer.zero_grad()
#         loss_dict = model(images, targets) 
        
#         # loss = sum(loss_dict.values())  # dict 형식이므로 `.values()` 사용 가능
#         # loss = sum(loss for loss in loss_dict.values())
#         loss = loss_dict['loss_classifier'] + 10*loss_dict['loss_box_reg'] + loss_dict['loss_objectness'] + 10*loss_dict['loss_rpn_box_reg']

#         loss.backward()
#         optimizer.step()
#         total_train_loss += loss.item()

#         train_loader_tqdm.set_postfix(loss=loss.item())

#         # Accuracy 계산 (폐렴이 포함된 이미지 비율)
#         for target in targets:
#             total_train_samples += len(target["labels"])
#             total_train_correct += target["labels"].sum().item()  # 모든 박스가 폐렴(1)이므로 단순 합

#     avg_train_loss = total_train_loss / len(train_loader)
#     train_accuracy = 100 * (total_train_correct / total_train_samples)

#     # Validation Step
#     model.eval()
#     total_val_loss = 0
#     total_val_correct = 0
#     total_val_samples = 0

#     val_loader_tqdm = tqdm(val_loader, desc="Validation", leave=False)
#     with torch.no_grad():
#         for images, targets in val_loader_tqdm:
#             images = [img.to(device) for img in images]
#             targets = [{k: v.to(device) for k, v in target.items()} for target in targets]

#             loss_dict = model(images, targets)  # 올바른 입력
#             #loss = sum(loss_dict.values())
#             print("test: ", loss_dict)
#             # loss = sum(loss for loss in loss_dict.values())
#             loss = loss_dict['loss_classifier'] + 10*loss_dict['loss_box_reg'] + loss_dict['loss_objectness'] + 10*loss_dict['loss_rpn_box_reg']


#             total_val_loss += loss.item()
#             val_loader_tqdm.set_postfix(loss=loss.item())

#             # Accuracy 계산
#             for target in targets:
#                 total_val_samples += len(target["labels"])
#                 total_val_correct += target["labels"].sum().item()

#     avg_val_loss = total_val_loss / len(val_loader)
#     val_accuracy = 100 * (total_val_correct / total_val_samples)

#     # Learning Rate Scheduler 업데이트
#     scheduler.step(avg_val_loss)

#     # Checkpoint 저장
#     if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         early_stopping_counter = 0
#         torch.save(model.state_dict(), "FasterRCNN_best.pth")
#         print(f"Best Model Saved! Epoch [{epoch+1}] | Val Loss: {avg_val_loss:.4f}")
#     else:
#         early_stopping_counter += 1

#     print(f"Epoch [{epoch+1}/{num_epochs}] - "
#           f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
#           f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%\n")
#     print("---------------------------------------------------------------")

#     # Early Stopping 확인
#     if early_stopping_counter >= early_stopping_patience:
#         print(f"Early Stopping at Epoch {epoch+1} | Best Val Loss: {best_val_loss:.4f}")
#         break  # 학습 중단

yunseo code

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision.ops import box_iou

# 옵티마이저 및 스케줄러 설정
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

num_epochs = 100
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
early_stopping_patience = 10  
early_stopping_counter = 0  
best_iou = 0  

train_losses = []  # Train Loss 저장 리스트

for epoch in range(num_epochs):
    print(f"\nEpoch [{epoch+1}/{num_epochs}] 시작\n")

    # Training Step
    model.train()
    total_train_loss = 0

    train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)
    for images, targets in train_loader_tqdm:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in target.items()} for target in targets] 

        optimizer.zero_grad()
        loss_dict = model(images, targets)  
        loss = loss_dict['loss_classifier'] + 10 * loss_dict['loss_box_reg'] + \
               loss_dict['loss_objectness'] + 10 * loss_dict['loss_rpn_box_reg']

        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        train_loader_tqdm.set_postfix(loss=loss.item())

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)  # Train Loss 저장

    # Validation Step
    model.eval()
    total_iou = 0
    total_detections = 0

    val_loader_tqdm = tqdm(val_loader, desc="Validation", leave=False)
    with torch.no_grad():
        for images, targets in val_loader_tqdm:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in target.items()} for target in targets]

            predictions = model(images)  

            for pred, target in zip(predictions, targets):
                pred_boxes = pred["boxes"]  
                gt_boxes = target["boxes"]  

                if len(pred_boxes) > 0 and len(gt_boxes) > 0:
                    ious = box_iou(pred_boxes, gt_boxes)  
                    max_iou_per_pred, _ = ious.max(dim=1)  
                    total_iou += max_iou_per_pred.sum().item()  
                    total_detections += len(pred_boxes)

    avg_iou = total_iou / total_detections if total_detections > 0 else 0

    # 체크포인트 저장 (최고 IoU 갱신 시)
    if avg_iou > best_iou:
        best_iou = avg_iou  
        early_stopping_counter = 0  # Early Stopping 리셋
        torch.save(model.state_dict(), "FasterRCNN_best_cossine.pth")  
        print(f"Best Model Saved! Epoch [{epoch+1}] | Best IoU: {best_iou:.4f}")

    else:
        early_stopping_counter += 1  # 개선되지 않으면 카운터 증가

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {avg_train_loss:.4f} | "
          f"Avg IoU: {avg_iou:.4f}\n")
    print("---------------------------------------------------------------")

    # Early Stopping 확인 (10번 연속 IoU 상승 실패 시 스탑)
    if early_stopping_counter >= early_stopping_patience:
        print(f"Early Stopping at Epoch {epoch+1} | Best IoU: {best_iou:.4f}")
        break  # 학습 중단

# Train Loss 그래프 출력
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, marker='o', linestyle='-')
plt.xlabel("Epoch")
plt.ylabel("Train Loss")
plt.title("Training Loss Over Epochs")
plt.grid(True)
plt.show()

In [64]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision.ops import box_iou

def test_model(model, test_loader, device):
    model.eval()
    total_iou = 0
    total_detections = 0
    all_ious = []
    all_precisions = []
    all_recalls = []
    
    with torch.no_grad():
        for images, targets in test_loader:
            images = [img.to(device) for img in images]
            print(images[0].shape)
            targets = [{k: v.to(device) for k, v in target.items()} for target in targets]
            
            predictions = model(images)  # 모델 예측
            print(predictions)
            for img, pred, target in zip(images, predictions, targets):
                pred_boxes = pred["boxes"].detach().cpu().numpy()
                gt_boxes = target["boxes"].detach().cpu().numpy()
                
                if len(pred_boxes) > 0 and len(gt_boxes) > 0:
                    ious = box_iou(torch.tensor(pred_boxes), torch.tensor(gt_boxes))
                    max_iou_per_pred, _ = ious.max(dim=1)
                    total_iou += max_iou_per_pred.sum().item()
                    total_detections += len(pred_boxes)
                    all_ious.extend(max_iou_per_pred.tolist())
                
                # # Precision, Recall 계산
                # tp = sum(max_iou_per_pred > 0.5)
                # fp = len(pred_boxes) - tp
                # fn = len(gt_boxes) - tp
                
                # precision = tp / (tp + fp + 1e-6)  # Precision 계산
                # recall = tp / (tp + fn + 1e-6)  # Recall 계산
                
                # all_precisions.append(precision)
                # all_recalls.append(recall)
                
                # Ground-Truth와 예측 박스 시각화 (Grayscale)
                visualize_boxes(img.cpu(), pred_boxes, gt_boxes)

    avg_iou = total_iou / total_detections if total_detections > 0 else 0
    mean_ap = np.mean(all_ious) if all_ious else 0
    mean_precision = np.mean(all_precisions) if all_precisions else 0
    mean_recall = np.mean(all_recalls) if all_recalls else 0
    
    print(f"Test IoU: {avg_iou:.4f}")
    print(f"Mean Average Precision (mAP): {mean_ap:.4f}")
    print(f"Mean Precision: {mean_precision:.4f}, Mean Recall: {mean_recall:.4f}")

def visualize_boxes(image, pred_boxes, gt_boxes):
    image = image.squeeze(0).numpy()  # Grayscale 변환
    plt.figure(figsize=(8, 8))
    plt.imshow(image, cmap='gray')
    
    for box in gt_boxes:
        x1, y1, x2, y2 = box
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='green', linewidth=2, label='GT'))
    
    for box in pred_boxes:
        x1, y1, x2, y2 = box
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2, label='Pred'))
    
    plt.legend(["Ground-Truth", "Prediction"])
    plt.show()

In [66]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision.ops import box_iou

def test_model(model, test_loader, device):
    model.eval()
    total_iou = 0
    total_detections = 0
    all_ious = []
    all_precisions = []
    all_recalls = []
    
    with torch.no_grad():
        for images, targets in test_loader:
            images = [img.to(device) for img in images]
            print("Image shape:", images[0].shape)
            targets = [{k: v.to(device) for k, v in target.items()} for target in targets]
            
            predictions = model(images)  # 모델 예측
            print("Predictions:", predictions)
            for img, pred, target in zip(images, predictions, targets):
                pred_boxes = pred["boxes"].detach().cpu().numpy()
                gt_boxes = target["boxes"].detach().cpu().numpy()
                
                if len(pred_boxes) > 0 and len(gt_boxes) > 0:
                    ious = box_iou(torch.tensor(pred_boxes), torch.tensor(gt_boxes))
                    max_iou_per_pred, _ = ious.max(dim=1)
                    total_iou += max_iou_per_pred.sum().item()
                    total_detections += len(pred_boxes)
                    all_ious.extend(max_iou_per_pred.tolist())
                
                # Precision, Recall 계산 (필요하면 주석 해제 후 조건 처리 추가)
                # if len(pred_boxes) > 0 and len(gt_boxes) > 0:
                #     tp = sum(max_iou_per_pred > 0.5)
                # else:
                #     tp = 0
                # fp = len(pred_boxes) - tp
                # fn = len(gt_boxes) - tp
                # precision = tp / (tp + fp + 1e-6) if (tp + fp) > 0 else 0.0
                # recall = tp / (tp + fn + 1e-6) if (tp + fn) > 0 else 0.0
                # all_precisions.append(precision)
                # all_recalls.append(recall)
                
                # Ground-Truth와 예측 박스 시각화 (그레이스케일)
                visualize_boxes(img.cpu(), pred_boxes, gt_boxes)

    avg_iou = total_iou / total_detections if total_detections > 0 else 0
    mean_ap = np.mean(all_ious) if all_ious else 0
    mean_precision = np.mean(all_precisions) if all_precisions else 0
    mean_recall = np.mean(all_recalls) if all_recalls else 0
    
    print(f"Test IoU: {avg_iou:.4f}")
    print(f"Mean Average Precision (mAP): {mean_ap:.4f}")
    print(f"Mean Precision: {mean_precision:.4f}, Mean Recall: {mean_recall:.4f}")

def visualize_boxes(image, pred_boxes, gt_boxes):
    # 이미지가 3채널인 경우, (H,W,C)로 변환 후 그레이스케일로 변환
    image = image.permute(1, 2, 0).numpy()  # (H, W, C)
    image_gray = np.mean(image, axis=2)  # 그레이스케일 (평균내기)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image_gray, cmap='gray')
    
    ax = plt.gca()
    # 한 번만 label을 설정하기 위한 플래그
    gt_label_added = False
    pred_label_added = False
    
    for box in gt_boxes:
        x1, y1, x2, y2 = box
        if not gt_label_added:
            ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                         fill=False, edgecolor='green', linewidth=2, label='GT'))
            gt_label_added = True
        else:
            ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                         fill=False, edgecolor='green', linewidth=2))
    
    for box in pred_boxes:
        x1, y1, x2, y2 = box
        if not pred_label_added:
            ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                         fill=False, edgecolor='red', linewidth=2, label='Pred'))
            pred_label_added = True
        else:
            ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                         fill=False, edgecolor='red', linewidth=2))
    
    plt.legend()
    plt.show()


In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchvision.ops import box_iou

# 모델 로드 함수
def load_model(checkpoint_path, device):
    model = fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2)  # 클래스 수 조정
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.to(device)
    model.eval()
    return model

# 모델 체크포인트 경로
checkpoint_path = "FasterRCNN_best_cossine.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 로드
model = load_model(checkpoint_path, device)

# 테스트 데이터셋 로드 (이 부분은 사용자 데이터셋에 맞게 조정 필요)
# 예시로 DataLoader만 생성 (실제 데이터셋을 사용해야 함)
# test_loader = DataLoader(your_test_dataset, batch_size=1, shuffle=False)

# 모델 평가 함수 실행
test_model(model, test_loader, device)


In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision.ops import box_iou
import torchvision

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 2)  # 배경 + 폐렴
model.load_state_dict(torch.load("FasterRCNN_best.pth", map_location=device))
model.to(device)
model.eval()

# IoU Threshold 설정
IOU_THRESHOLD = 0.5

# 평가 변수 초기화
total_loss = 0
total_loss_classifier = 0
total_loss_box_reg = 0
total_loss_objectness = 0
total_loss_rpn_box_reg = 0
total_iou = 0
total_detections = 0

test_loader_tqdm = tqdm(test_loader, desc="Testing", leave=False)

# Test Loop 시작
with torch.no_grad():
    for images, targets in test_loader_tqdm:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in target.items()} for target in targets]

        # 손실 계산
        model.train()  # Training Mode로 변경 (손실 계산을 위해)
        loss_dict = model(images, targets)  # 손실 반환
        model.eval()  # 다시 평가 모드로 변경

        # 손실 값 개별 저장
        loss_classifier = loss_dict["loss_classifier"].item()
        loss_box_reg = loss_dict["loss_box_reg"].item()
        loss_objectness = loss_dict["loss_objectness"].item()
        loss_rpn_box_reg = loss_dict["loss_rpn_box_reg"].item()

        # 총 손실 계산
        total_sample_loss = loss_classifier + loss_box_reg + loss_objectness + loss_rpn_box_reg

        # IoU 계산
        predictions = model(images)
        for pred, target in zip(predictions, targets):
            pred_boxes = pred["boxes"].cpu().numpy()
            gt_boxes = target["boxes"].cpu().numpy()

            if len(pred_boxes) > 0 and len(gt_boxes) > 0:
                ious = box_iou(torch.tensor(pred_boxes), torch.tensor(gt_boxes))
                max_iou_per_pred, _ = ious.max(dim=1)

                total_iou += max_iou_per_pred.sum().item()
                total_detections += len(pred_boxes)

        # 손실값 저장
        total_loss += total_sample_loss
        total_loss_classifier += loss_classifier
        total_loss_box_reg += loss_box_reg
        total_loss_objectness += loss_objectness
        total_loss_rpn_box_reg += loss_rpn_box_reg

        # 손실값 및 IoU 출력
        print(f"Loss Breakdown - Classifier: {loss_classifier:.4f}, Box Reg: {loss_box_reg:.4f}, "
              f"Objectness: {loss_objectness:.4f}, RPN Box Reg: {loss_rpn_box_reg:.4f}, "
              f"Total Loss: {total_sample_loss:.4f}")

# 테스트 결과 출력
avg_loss = total_loss / len(test_loader)
avg_loss_classifier = total_loss_classifier / len(test_loader)
avg_loss_box_reg = total_loss_box_reg / len(test_loader)
avg_loss_objectness = total_loss_objectness / len(test_loader)
avg_loss_rpn_box_reg = total_loss_rpn_box_reg / len(test_loader)
avg_iou = total_iou / total_detections if total_detections > 0 else 0

print("\nTest Results:")
print(f"Average Total Loss: {avg_loss:.4f}")
print(f"Average Classifier Loss: {avg_loss_classifier:.4f}")
print(f"Average Box Regression Loss: {avg_loss_box_reg:.4f}")
print(f"Average Objectness Loss: {avg_loss_objectness:.4f}")
print(f"Average RPN Box Regression Loss: {avg_loss_rpn_box_reg:.4f}")
print(f"Test IoU: {avg_iou:.4f}")
